# PyTorch MobileNetV3-Small — End-to-End (Fast)

# Sea Animals Image Classification — End-to-End
This notebook runs **split → train → evaluate → predict** for one algorithm.
**Expected folder on your machine** (run this notebook from your project root):

```
Sea-Animals-Image-DatasetProject/
├── Sea Animals Image Dataset/   # raw classes folders
├── FishDataset/                 # will be created (train/val)
└── models/                      # will be created (artifacts)
```

> If your raw dataset folder is named differently (e.g., `Sea_Animals_Image_Dataset`), edit the `DATASET_CANDIDATE_NAMES` list in the split cell.


## 0) Optional installs (run if needed)

In [1]:
## !pip install torch torchvision matplotlib scikit-learn pillow

## 1) Setup paths

In [2]:
# Paths and basic setup
from pathlib import Path
import os, shutil, random, time
from sklearn.model_selection import train_test_split

ROOT = Path.cwd()
RAW_DIR = None
DATASET_CANDIDATE_NAMES = [
    "Sea Animals Image Dataset",
    "Sea_Animals_Image_Dataset",
    "SeaAnimalsImageDataset",
]

for name in DATASET_CANDIDATE_NAMES:
    cand = ROOT / name
    if cand.exists() and cand.is_dir():
        RAW_DIR = cand
        break

if RAW_DIR is None:
    raise FileNotFoundError(
        "Raw dataset folder not found. Create a folder named one of: "
        + ", ".join(DATASET_CANDIDATE_NAMES)
        + " inside your project root."
    )

FISHDATASET = ROOT / "FishDataset"
TRAIN_DIR = FISHDATASET / "train"
VAL_DIR = FISHDATASET / "val"
MODELS_DIR = ROOT / "models"
MODELS_DIR.mkdir(exist_ok=True, parents=True)

print("Project root:", ROOT)
print("Raw dataset :", RAW_DIR)
print("FishDataset :", FISHDATASET)
print("Models dir  :", MODELS_DIR)


Project root: f:\Git\Sea Animals Image DatasetProject
Raw dataset : f:\Git\Sea Animals Image DatasetProject\Sea Animals Image Dataset
FishDataset : f:\Git\Sea Animals Image DatasetProject\FishDataset
Models dir  : f:\Git\Sea Animals Image DatasetProject\models


## 2) Split dataset (80/20)

In [3]:
# Split raw dataset into train/val (80/20) — idempotent
import os, shutil
from sklearn.model_selection import train_test_split

TRAIN_DIR.mkdir(parents=True, exist_ok=True)
VAL_DIR.mkdir(parents=True, exist_ok=True)

classes = [d for d in RAW_DIR.iterdir() if d.is_dir()]
if not classes:
    raise RuntimeError(f"No class folders found under {RAW_DIR}")

count_summary = []
for cls_path in classes:
    cls = cls_path.name
    images = [p for p in cls_path.iterdir() if p.suffix.lower() in {".jpg",".jpeg",".png"}]
    if not images:
        print("Skip (no images):", cls)
        continue
    from sklearn.model_selection import train_test_split
    train, val = train_test_split(images, test_size=0.2, random_state=42)
    (TRAIN_DIR/cls).mkdir(parents=True, exist_ok=True)
    (VAL_DIR/cls).mkdir(parents=True, exist_ok=True)
    for src in train:
        dst = TRAIN_DIR/cls/src.name
        if not dst.exists():
            shutil.copy2(src, dst)
    for src in val:
        dst = VAL_DIR/cls/src.name
        if not dst.exists():
            shutil.copy2(src, dst)
    count_summary.append((cls, len(train), len(val)))
    print(f"Class {cls}: {len(train)} train / {len(val)} val")

print("✅ Split done → FishDataset/train & FishDataset/val")
count_summary


Class Clams: 397 train / 100 val
Class Corals: 400 train / 100 val
Class Crabs: 399 train / 100 val
Class Dolphin: 625 train / 157 val
Class Eel: 397 train / 100 val
Class Fish: 395 train / 99 val
Class Jelly Fish: 676 train / 169 val
Class Lobster: 399 train / 100 val
Class Nudibranchs: 400 train / 100 val
Class Octopus: 449 train / 113 val
Class Otter: 400 train / 100 val
Class Penguin: 385 train / 97 val
Class Puffers: 424 train / 107 val
Class Sea Rays: 413 train / 104 val
Class Sea Urchins: 463 train / 116 val
Class Seahorse: 382 train / 96 val
Class Seal: 331 train / 83 val
Class Sharks: 472 train / 118 val
Class Shrimp: 390 train / 98 val
Class Squid: 386 train / 97 val
Class Starfish: 399 train / 100 val
Class Turtle_Tortoise: 216 train / 55 val
✅ Split done → FishDataset/train & FishDataset/val


[('Clams', 397, 100),
 ('Corals', 400, 100),
 ('Crabs', 399, 100),
 ('Dolphin', 625, 157),
 ('Eel', 397, 100),
 ('Fish', 395, 99),
 ('Jelly Fish', 676, 169),
 ('Lobster', 399, 100),
 ('Nudibranchs', 400, 100),
 ('Octopus', 449, 113),
 ('Otter', 400, 100),
 ('Penguin', 385, 97),
 ('Puffers', 424, 107),
 ('Sea Rays', 413, 104),
 ('Sea Urchins', 463, 116),
 ('Seahorse', 382, 96),
 ('Seal', 331, 83),
 ('Sharks', 472, 118),
 ('Shrimp', 390, 98),
 ('Squid', 386, 97),
 ('Starfish', 399, 100),
 ('Turtle_Tortoise', 216, 55)]

## 3) Train

In [ ]:
# Train MobileNetV3-Small
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
print("Device:", device)

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform_train)
val_dataset   = datasets.ImageFolder(VAL_DIR,   transform=transform_val)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)

num_classes = len(train_dataset.classes)

model = models.mobilenet_v3_small(weights="IMAGENET1K_V1")
for p in model.parameters():
    p.requires_grad = False

model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=2e-4)

EPOCHS = 8
train_acc, val_acc = [], []
best = 0.0

for epoch in range(EPOCHS):
    model.train()
    correct, total = 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    tr = 100*correct/total
    train_acc.append(tr)

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
    va = 100*correct/total
    val_acc.append(va)
    print(f"Epoch {epoch+1}/{EPOCHS} → Train={tr:.2f}% | Val={va:.2f}%")
    if va > best:
        best = va
        torch.save(model.state_dict(), MODELS_DIR / "mobilenetv3_best.pth")

plt.figure()
plt.plot(train_acc, label="Train")
plt.plot(val_acc, label="Val")
plt.title("Accuracy")
plt.legend()
plt.savefig(MODELS_DIR / "mobilenetv3_accuracy.png")
plt.show()

print("✅ Best Val Acc:", best)

Device: cpu
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\FURY/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth


100.0%


Epoch 1/8 → Train=36.84% | Val=54.35%


## 4) Predict

In [ ]:
# Predict — MobileNetV3
import torch, random
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
classes = [d.name for d in VAL_DIR.iterdir() if d.is_dir()]
num_classes = len(classes)

model = models.mobilenet_v3_small(weights=None)
model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, num_classes)
model.load_state_dict(torch.load(MODELS_DIR / "mobilenetv3_best.pth", map_location=device))
model.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

sample = random.choice(list(VAL_DIR.glob("*/*.jpg")))
img = Image.open(sample).convert("RGB")
x = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    logits = model(x)
    probs = torch.nn.functional.softmax(logits, dim=1)[0]
    idx = int(torch.argmax(probs))
    conf = float(probs[idx])*100
label = classes[idx]

print(f"✅ Prediction: {label} ({conf:.2f}%) — {sample}")
plt.imshow(img)
plt.axis("off")
plt.title(f"{label} ({conf:.2f}%)")
plt.show()